In [2]:
# https://github.com/philipperemy/name-dataset
# !pip install names-dataset
#from names_dataset import NameDataset
#nd = NameDataset()
import numpy as np
import os
import pandas as pd
import pickle
from scipy import stats
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
from pathlib import Path

In [3]:
countries = ['FI', 'HR', 'IT', 'GB', 'DK', 'FR']
country2id = dict()
id2country = dict()
for i, file in enumerate(os.listdir('data')):
    t = file.split('.')
    country2id[t[0]] = i
country_ids = [country2id[country] for country in countries]
for country in countries:
    id2country[country2id[country]] = country

In [4]:
#name_corpus = []
surname_corpus = []

for file in tqdm(os.listdir('parquet')):
    df = pd.read_parquet(f'parquet/{file}', columns=['lastName'])
    counts = df['lastName'].value_counts()[:10_000].reset_index()
    counts['index'] = counts['index'].str.replace(" ", "-")
    temp = (counts['index']+" ").str.repeat(counts.lastName)
    #name_corpus.append(temp.to_frame().T.apply(" ".join, axis=1))
    surname_corpus.append(temp.to_frame().T.apply(" ".join, axis=1))


FileNotFoundError: [WinError 3] Den angivne sti blev ikke fundet: 'parquet'

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(pd.DataFrame(surname_corpus).squeeze().values.tolist())

In [68]:
outfile = open('surname_vectorizer.pickle','wb')
pickle.dump(vectorizer,outfile)
outfile.close()
outfile = open('surname_matrix.pickle','wb')
pickle.dump(X,outfile)
outfile.close()

In [5]:
with open(r'vectorizer', 'rb') as f:
    vectorizer = pickle.load(f)
with open(r'matrix', 'rb') as f:
    X = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'surname_vectorizer.pickle'

In [6]:
name_array = vectorizer.get_feature_names_out()
df = pd.DataFrame(X.A)

In [24]:
def name_mapper(n): return name_array[int(n)]
dfCountries = df[df.index.isin(country_ids)]
dfCountries = dfCountries.loc[:, (dfCountries != 0).any(axis=0)]

In [38]:
country_series = []
n = 1000
for id in country_ids:
    tempdf = dfCountries.loc[id].sort_values(ascending=False)
    tempdf = tempdf.rename(index=name_mapper)
    country_series.append(tempdf.iloc[:n].reset_index().rename(columns={'index':id2country[id]})[id2country[id]])
names_df = pd.concat(country_series,axis=1)

In [43]:
names_df['IT']

0        giuseppe
1       francesco
2      alessandro
3         stefano
4            luca
          ...    
995        morana
996         molly
997         moona
998        mourad
999        vibeke
Name: IT, Length: 1000, dtype: object

In [ ]:
print(dfCountries[800:900].to_string())

In [ ]:
print(nd.search('Giuseppe'))

In [ ]:
print(nd.search('Giuseppe')['first_name'])

In [ ]:
c = 'IT'
print(nd.get_top_names(n=100, gender='Male',country_alpha2=c)[c]['M'][-20:])

In [ ]:
dataset = {c:{"Female": nd.get_top_names(n=100,gender="Female",country_alpha2=c)[c]["F"],
                        "Male": nd.get_top_names(n=100,gender="Male",country_alpha2=c)[c]["M"]} for c in countries}

In [7]:
file

'AE.csv'

In [6]:
data_folder = 'data'
parquet_folder = Path('parquet')
parquet_folder.mkdir(exist_ok=True)
for file in tqdm(os.listdir(data_folder)):
   out_file = parquet_folder / file
   if out_file.exists():
       continue

   df = pd.read_csv(f'{data_folder}/{file}', header=None, names=["firstName","lastName","gender","country"], dtype="string", encoding='utf8')
   df.to_parquet(out_file)

  0%|          | 0/105 [00:00<?, ?it/s]


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa2 in position 12: invalid start byte

In [ ]:
#name_vocabulary = list(set((item for sublist in name_corpus for item in sublist)))
#surname_vocabulary = list(set((item for sublist in surname_corpus for item in sublist)))

In [ ]:
'''from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
pipe = Pipeline([('count', CountVectorizer(vocabulary=name_vocabulary)),
                  ('tfid', TfidfTransformer())]).fit(name_corpus)
pipe['count'].transform(corpus).toarray()'''